In [1]:
# Importar librerías necesarias
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import numpy as np
import os
import cv2
import pandas as pd

# Rutas de Datos
ruta_train_csv = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\train.csv'
ruta_train_img = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\train'
ruta_modelo = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\dsft2409_mikeltelo\projects\kaggle_clip_counting\modelo_clip_count_final.keras'

# Cargar el modelo preentrenado
modelo = load_model(ruta_modelo)

# Mostrar resumen del modelo
modelo.summary()

# Función para transformar imágenes
def transformar_imagen(ruta):
    img = cv2.imread(ruta)
    if img is None:
        raise FileNotFoundError(f"No se pudo cargar la imagen en la ruta: {ruta}")
    
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask_roja1 = cv2.inRange(img_hsv, (0, 50, 50), (10, 255, 255))
    mask_roja2 = cv2.inRange(img_hsv, (170, 50, 50), (180, 255, 255))
    mask_roja = cv2.bitwise_or(mask_roja1, mask_roja2)
    img[mask_roja > 0] = (255, 0, 0)
    
    b = img[:, :, 0]
    _, thresh = cv2.threshold(b, 225, 255, cv2.THRESH_BINARY)
    thresh_invertida = 255 - thresh
    
    img_final = cv2.resize(thresh_invertida, (256, 256))
    img_final = img_final.astype(np.float32) / 255.0
    
    return img_final

# Cargar y transformar datos de entrenamiento
df_train = pd.read_csv(ruta_train_csv)
X = []
y = []

for index, row in df_train.iterrows():
    ruta_img = os.path.join(ruta_train_img, f"clips-{row['id']}.png")
    if os.path.exists(ruta_img):
        img = transformar_imagen(ruta_img)
        X.append(img)
        y.append(row['clip_count'])

# Convertir a NumPy y preparar para entrenamiento
X = np.array(X, dtype=np.float32).reshape(-1, 256, 256, 1)
y = np.array(y, dtype=np.int32)

# Dividir datos
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Compilar modelo para ajuste fino con tasa de aprendizaje baja
modelo.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Tasa de aprendizaje muy baja
    loss='mean_squared_error',
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    min_lr=1e-6,
    verbose=1
)

# Entrenamiento de ajuste fino
historial = modelo.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=64,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Evaluar el modelo ajustado
loss, rmse = modelo.evaluate(X_val, y_val)
print(f"Pérdida final en validación: {loss}, RMSE: {rmse}")

# Guardar el modelo ajustado
ruta_modelo_ajustado = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\dsft2409_mikeltelo\projects\kaggle_clip_counting\modelo_clip_count_ajustado.keras'
modelo.save(ruta_modelo_ajustado)
print(f"Modelo ajustado guardado en: {ruta_modelo_ajustado}")


C:\Users\mikel\AppData\Roaming\Python\Python39\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_25 (Conv2D)              │ (None, 254, 254, 16)   │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_25 (MaxPooling2D) │ (None, 127, 127, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 125, 125, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_26 (MaxPooling2D) │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 123008)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 32)             │     3,936,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,882,244 (30.07 MB)

 Trainable params: 3,941,121 (15.03 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,941,123 (15.03 MB)

Epoch 1/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 61s 306ms/step - loss: 0.1490 - root_mean_squared_error: 0.3859 - val_loss: 4.0877 - val_root_mean_squared_error: 2.0218 - learning_rate: 1.0000e-05
Epoch 2/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 55s 294ms/step - loss: 0.1372 - root_mean_squared_error: 0.3704 - val_loss: 4.0803 - val_root_mean_squared_error: 2.0200 - learning_rate: 1.0000e-05
Epoch 3/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 56s 300ms/step - loss: 0.1363 - root_mean_squared_error: 0.3691 - val_loss: 4.0772 - val_root_mean_squared_error: 2.0192 - learning_rate: 1.0000e-05
Epoch 4/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 57s 301ms/step - loss: 0.1326 - root_mean_squared_error: 0.3641 - val_loss: 4.0803 - val_root_mean_squared_error: 2.0200 - learning_rate: 1.0000e-05
Epoch 5/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 57s 304ms/step - loss: 0.1263 - root_mean_squared_error: 0.3553 - val_loss: 4.0764 - val_root_mean_squared_error: 2.0190 - learning_rate: 1.0000e-05
Epoch 6/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 55s 294ms/step 

In [2]:
# Evaluar el modelo ajustado
loss, rmse = modelo.evaluate(X_val, y_val)
print(f"Pérdida final en validación: {loss}, RMSE: {rmse}")


94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - loss: 4.0873 - root_mean_squared_error: 2.0214
Pérdida final en validación: 4.069591045379639, RMSE: 2.0173227787017822


In [3]:
# Realizar predicciones finales
predicciones = modelo.predict(X_val).flatten()
predicciones = np.round(predicciones).astype(int)
predicciones = np.clip(predicciones, 0, 75)

94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step
